# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 0: Business Question

 For this project, you will be acting as a consultant for a fictional real-estate investment firm. The firm has asked you what seems like a simple question:

    What are the top 5 best zip codes for us to invest in?

This may seem like a simple question at first glance, but there's more than a little ambiguity here that you'll have to think through in order to provide a solid recommendation. Should your recommendation be focused on profit margins only? What about risk? What sort of time horizon are you predicting against? Your recommendation will need to detail your rationale and answer any sort of lingering questions like these in order to demonstrate how you define "best".

# Step 1: Load the Data/Filtering for Chosen Zipcodes

import libraries

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px

initial print

In [2]:
df = pd.read_csv("data/zillow_data.csv")
nh_df = pd.read_csv("data/neighborhood_data.csv")
nh_df.head(10)
#df.info()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31
0,274772,0,Northeast Dallas,Neighborhood,TX,TX,Dallas,Dallas-Fort Worth-Arlington,Dallas County,135822.0,...,340469.0,342908.0,344906.0,347240.0,351339.0,355704.0,360122.0,361710.0,363919.0,365908.0
1,112345,1,Maryvale,Neighborhood,AZ,AZ,Phoenix,Phoenix-Mesa-Scottsdale,Maricopa County,NaN,...,217419.0,220343.0,223627.0,227398.0,230851.0,234753.0,238070.0,241632.0,244012.0,245332.0
2,192689,2,Paradise,Neighborhood,NV,NV,Las Vegas,Las Vegas-Henderson-Paradise,Clark County,141784.0,...,279695.0,280955.0,283492.0,286903.0,289515.0,291818.0,293802.0,296043.0,298533.0,301325.0
3,270958,3,Upper West Side,Neighborhood,NY,NY,New York,New York-Newark-Jersey City,New York County,236241.0,...,1150808.0,1155906.0,1161040.0,1166959.0,1162385.0,1157878.0,1155829.0,1150443.0,1155436.0,1157749.0
4,118208,4,South Los Angeles,Neighborhood,CA,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,136341.0,...,554377.0,559670.0,567182.0,574883.0,581415.0,586627.0,590405.0,594516.0,599448.0,605661.0
5,270957,5,Upper East Side,Neighborhood,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,1077704.0,1082774.0,1088528.0,1089680.0,1096836.0,1096908.0,1098618.0,1089350.0,1086647.0,1087600.0
6,192820,6,Sunrise Manor,Neighborhood,NV,NV,Las Vegas,Las Vegas-Henderson-Paradise,Clark County,NaN,...,241220.0,242364.0,244566.0,247673.0,250067.0,252235.0,254057.0,256122.0,258354.0,260858.0
7,192796,7,Spring Valley,Neighborhood,NV,NV,Las Vegas,Las Vegas-Henderson-Paradise,Clark County,151473.0,...,300381.0,301544.0,304112.0,307643.0,310236.0,312423.0,314256.0,316421.0,318903.0,321736.0
8,268496,8,Southeast Los Angeles,Neighborhood,CA,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,NaN,...,450242.0,453694.0,458551.0,463980.0,467922.0,472629.0,477045.0,481464.0,485057.0,488179.0
9,194430,9,East New York,Neighborhood,NY,NY,New York,New York-Newark-Jersey City,Kings County,155510.0,...,523580.0,526890.0,530698.0,533772.0,537217.0,540039.0,541997.0,543028.0,544048.0,543124.0


separate chicago zips

In [3]:
df = df.loc[df['Metro'] == 'Chicago']
nh_df = nh_df.loc[nh_df['City'] == 'Chicago']
nh_df.head(10)
#nh_df.info()


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31
52,269592,53,Logan Square,Neighborhood,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,153252.0,...,447271.0,450368.0,453928.0,458104.0,462215.0,465269.0,468493.0,471165.0,474865.0,477745.0
87,403117,88,Little Village,Neighborhood,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,NaN,...,177273.0,179695.0,179917.0,179802.0,179935.0,179985.0,180271.0,180635.0,182971.0,183939.0
104,403169,106,West Rogers Park,Neighborhood,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,135374.0,...,310971.0,313105.0,315046.0,317957.0,320474.0,322858.0,324789.0,326025.0,327693.0,329638.0
141,403120,144,South Austin,Neighborhood,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,NaN,...,192147.0,195018.0,198106.0,201893.0,206552.0,211075.0,215168.0,218518.0,223578.0,228216.0
151,269566,154,Albany Park,Neighborhood,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,134894.0,...,345340.0,346957.0,349330.0,353102.0,355953.0,357524.0,357603.0,358035.0,361003.0,364556.0
156,269609,159,Uptown,Neighborhood,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,118402.0,...,304312.0,305517.0,307116.0,309456.0,311661.0,313599.0,315295.0,317073.0,319192.0,320845.0
160,269589,163,Lake View,Neighborhood,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,219907.0,...,539705.0,541323.0,543719.0,547161.0,550532.0,553354.0,556021.0,558278.0,561010.0,562892.0
168,269605,171,Rogers Park,Neighborhood,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,104550.0,...,220928.0,222099.0,223516.0,225480.0,227250.0,228823.0,230244.0,231633.0,233280.0,234766.0
202,269571,206,Gresham,Neighborhood,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,99648.0,...,153501.0,155769.0,158455.0,161330.0,164813.0,168101.0,171204.0,173176.0,177119.0,181122.0
203,273040,207,Brighton Park,Neighborhood,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,101526.0,...,212135.0,214417.0,216677.0,219210.0,221223.0,223545.0,225501.0,227418.0,229710.0,231905.0


hand-entered zips for city proper

In [4]:
city_zips = [60626, 60645, 60646, 60631, 60660, 
             60659, 60640, 60625, 60630, 60656, 
             60634, 60641, 60618, 60613, 60657, 
             60707, 60639, 60647, 60614, 60666, 
             60611, 60610, 60622, 60651, 60644, 
             60624, 60612, 60607, 60661, 60606, 
             60601, 60603, 60604, 60654, 60616, 
             60608, 60623, 60632, 60609, 60653, 
             60615, 60638, 60636, 60637, 60629, 
             60652, 60620, 60619, 60643, 60655, 
             60628, 60827, 60649, 60617, 60633, 
             60619, 60621, 60661]

print(len(city_zips))
print(len(set(city_zips)))
city_zips = list(set(city_zips))

58
56


# Step 2: Data Preprocessing

adjusted get_datetimes function to work on different types of dataframes

In [5]:
def get_datetimes(df, startcol):
    return pd.to_datetime(df.columns.values[startcol:], format='%Y-%m')

get_datetimes(df, 7)
get_datetimes(nh_df, 9)

DatetimeIndex(['1996-01-31', '1996-02-29', '1996-03-31', '1996-04-30',
               '1996-05-31', '1996-06-30', '1996-07-31', '1996-08-31',
               '1996-09-30', '1996-10-31',
               ...
               '2020-06-30', '2020-07-31', '2020-08-31', '2020-09-30',
               '2020-10-31', '2020-11-30', '2020-12-31', '2021-01-31',
               '2021-02-28', '2021-03-31'],
              dtype='datetime64[ns]', length=303, freq=None)

dealing with null values

In [6]:
pd.set_option('display.max_rows', None)
nh_df.isna().any()

RegionID      False
SizeRank      False
RegionName    False
RegionType    False
StateName     False
State         False
City          False
Metro         False
CountyName    False
1996-01-31     True
1996-02-29     True
1996-03-31     True
1996-04-30     True
1996-05-31     True
1996-06-30     True
1996-07-31     True
1996-08-31     True
1996-09-30     True
1996-10-31     True
1996-11-30     True
1996-12-31     True
1997-01-31     True
1997-02-28     True
1997-03-31     True
1997-04-30     True
1997-05-31     True
1997-06-30     True
1997-07-31     True
1997-08-31     True
1997-09-30     True
1997-10-31     True
1997-11-30     True
1997-12-31     True
1998-01-31     True
1998-02-28     True
1998-03-31     True
1998-04-30     True
1998-05-31     True
1998-06-30     True
1998-07-31     True
1998-08-31     True
1998-09-30     True
1998-10-31     True
1998-11-30     True
1998-12-31     True
1999-01-31     True
1999-02-28     True
1999-03-31     True
1999-04-30     True
1999-05-31     True


In [7]:
df.shape[1] - df.count(axis=1)

0          0
3          0
7          0
16         0
19         0
40         0
48         0
50         0
74         0
79         0
105      207
150        0
157        0
168        0
169        0
215        0
231        0
284        0
318        0
324        0
332        0
340        0
347        0
368        0
383        0
457        0
497        0
554        0
625        0
638        0
658        0
725        0
739        0
792        0
882        0
948        0
1005       0
1012       0
1089       0
1119       0
1152       0
1161       0
1192       0
1205       0
1298       0
1332       0
1334       0
1360       0
1392       0
1409       0
1414       0
1433       0
1454       0
1457       0
1487       0
1491       0
1496       0
1527       0
1562       0
1570       0
1594       0
1623       0
1641       0
1667       0
1686       0
1705       0
1721       0
1727       0
1864       0
1870       0
1883       0
1943       0
1950       0
1990       0
2048       0
2052       0
2114       0

strategy: melt here and then have option to use for visualizations

In [18]:
def melt_data(df):
    melted = pd.melt(df, 
                     id_vars=['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'], 
                     var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted
melted = melt_data(df)
melted.head(50)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,time,value
0,84654,60657,Chicago,IL,Chicago,Cook,1,1996-04-01,334200.0
1,84616,60614,Chicago,IL,Chicago,Cook,4,1996-04-01,498100.0
2,84640,60640,Chicago,IL,Chicago,Cook,8,1996-04-01,216500.0
3,84646,60647,Chicago,IL,Chicago,Cook,17,1996-04-01,122700.0
4,84620,60618,Chicago,IL,Chicago,Cook,20,1996-04-01,142600.0
5,84615,60613,Chicago,IL,Chicago,Cook,41,1996-04-01,297900.0
6,84630,60629,Chicago,IL,Chicago,Cook,49,1996-04-01,93400.0
7,84627,60625,Chicago,IL,Chicago,Cook,51,1996-04-01,148900.0
8,84621,60619,Chicago,IL,Chicago,Cook,75,1996-04-01,76400.0
9,84612,60610,Chicago,IL,Chicago,Cook,80,1996-04-01,435200.0


# Step 3: EDA and Visualization

In [8]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

plt.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

In [9]:
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
7,84640,60640,Chicago,IL,Chicago,Cook,8,216500.0,216700.0,216900.0,...,798000,787100,776100,774900,777900,777900,778500,780500,782800,782800
16,84646,60647,Chicago,IL,Chicago,Cook,17,122700.0,122800.0,122800.0,...,459600,461400,464300,466500,467900,470600,474500,475100,472600,470200
19,84620,60618,Chicago,IL,Chicago,Cook,20,142600.0,143100.0,143400.0,...,462700,461200,459900,459200,458700,457900,457400,459000,462500,464300


In [10]:
info_cols = list(df.columns[1:7])
info_cols

for col in info_cols:
    print(col)
    print(df[col].describe())

RegionName
count      325.000000
mean     58851.895385
std       4110.512992
min      46303.000000
25%      60070.000000
50%      60194.000000
75%      60513.000000
max      60805.000000
Name: RegionName, dtype: float64
City
count         325
unique        242
top       Chicago
freq           41
Name: City, dtype: object
State
count     325
unique      3
top        IL
freq      285
Name: State, dtype: object
Metro
count         325
unique          1
top       Chicago
freq          325
Name: Metro, dtype: object
CountyName
count      325
unique      13
top       Cook
freq       140
Name: CountyName, dtype: object
SizeRank
count      325.000000
mean      5457.323077
std       3804.744361
min          1.000000
25%       2289.000000
50%       4985.000000
75%       7968.000000
max      14605.000000
Name: SizeRank, dtype: float64


In [11]:
city_df = df.loc[df['City'] == 'Chicago']
city_df.info()
for col in info_cols:
    print(col)
    print(city_df[col].describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41 entries, 0 to 7863
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 87.4+ KB
RegionName
count       41.000000
mean     60633.097561
std         16.887577
min      60605.000000
25%      60618.000000
50%      60633.000000
75%      60647.000000
max      60660.000000
Name: RegionName, dtype: float64
City
count          41
unique          1
top       Chicago
freq           41
Name: City, dtype: object
State
count     41
unique     1
top       IL
freq      41
Name: State, dtype: object
Metro
count          41
unique          1
top       Chicago
freq           41
Name: Metro, dtype: object
CountyName
count       41
unique       1
top       Cook
freq        41
Name: CountyName, dtype: object
SizeRank
count      41.000000
mean     1266.536585
std      1660.637153
min         1.000000
25%       106.000000
50%       498.000000
75%      1991.000000
max      7864.000000
Name: SizeRank, dtype: floa

Time Series Aspects

In [12]:
#stationary

In [13]:
#seasonality

In [14]:
#autocorrelation

# Step 4: Reshape from Wide to Long Format

fixed melt function 

check

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,time,value
0,84654,60657,Chicago,IL,Chicago,Cook,1,1996-04-01,334200.0
1,84616,60614,Chicago,IL,Chicago,Cook,4,1996-04-01,498100.0
2,84640,60640,Chicago,IL,Chicago,Cook,8,1996-04-01,216500.0
3,84646,60647,Chicago,IL,Chicago,Cook,17,1996-04-01,122700.0
4,84620,60618,Chicago,IL,Chicago,Cook,20,1996-04-01,142600.0
5,84615,60613,Chicago,IL,Chicago,Cook,41,1996-04-01,297900.0
6,84630,60629,Chicago,IL,Chicago,Cook,49,1996-04-01,93400.0
7,84627,60625,Chicago,IL,Chicago,Cook,51,1996-04-01,148900.0
8,84621,60619,Chicago,IL,Chicago,Cook,75,1996-04-01,76400.0
9,84612,60610,Chicago,IL,Chicago,Cook,80,1996-04-01,435200.0


# Step 5: ARIMA Modeling

# Step 6: Interpreting Results